## Extract cloud image

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
from PIL import Image

processed_customers_metadata_df = pd.read_csv("configs/processed_customers_metadata.csv")
processed_customers_metadata_df

,customer,lat,lng,row_idx_TIFF,col_idx_TIFF,lat_TIFF,lng_TIFF
0,ee_building,13.736774,100.532122,847,864,13.736502,100.52990
1,FN_huahin,12.666830,99.952905,908,832,12.662679,99.95509


In [7]:
collected_df = pd.DataFrame(columns=['customer', 'Datetime', 'cloud_index'])
k = 0
ch = 0
image_path = './cloud_images'
cloud_image_list = sorted(os.listdir(image_path))
cloud_image_list 

for filename in cloud_image_list :
    I = Image.open(os.path.join(image_path, filename))
    imarray = np.array(I)
    time = datetime.strptime(filename.split('_')[-1].split(".")[0], '%Y%m%d%H%M') + timedelta(hours=7)

    
    cloud_index = imarray[processed_customers_metadata_df['row_idx_TIFF'], processed_customers_metadata_df['col_idx_TIFF'], ch] / 255
    df_1file = pd.DataFrame({
        'customer': processed_customers_metadata_df['customer'],
        'Datetime': time,
        'cloud_index': cloud_index
    })
    df_1file['Datetime'] = df_1file['Datetime'].dt.tz_localize("Asia/Bangkok")
        
    collected_df = pd.concat([collected_df, df_1file], ignore_index=True)


/var/folders/6m/8y18j89n795_15zj80c2rjb40000gn/T/ipykernel_70887/1717047557.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  collected_df = pd.concat([collected_df, df_1file], ignore_index=True)


## Save output separately for each customer

In [9]:
for customer in processed_customers_metadata_df['customer'].unique():
    customer_df = collected_df[collected_df['customer']==customer]
    customer_df.to_csv(f"extracted_cloud_output/{customer}_extracted_cloud.csv", index=False)